In [1267]:
import pandas as pd
import numpy as np
import random

import os

In [1268]:
"""
Simulation of the testing process of the single brick batches
"""



date = 240826
d = 5
brightness_rank = 10
k_colour = 3
k = 10
list_index = 1

file_path = f"C:\\Users\\david\\OneDrive\\Documents\\01_Architektur\\02_MA Architektur\\TUM\\2024_Sommersemester\\01_THESIS\\02_MA_Thesis\\02_Listing\\01_Visual_Studio\\240816_Simulation\\02-2_Generate_Batches\\{date}_{d}_{brightness_rank}_k-{k_colour}-{k}_{list_index}.csv"
name = f"{d}_k-{k_colour}_{brightness_rank}-{list_index}"
print(name)
try:
    df_batch = pd.read_csv(file_path, sep=",", encoding="utf-8")
    print("utf encoder :")
except UnicodeDecodeError:
    df_batch = pd.read_csv(file_path, sep=",", encoding="ISO-8859-1")
    print("iso encoder :")


sample_size = 6
sample_size = min(sample_size, len(df_batch))
samples = df_batch.sample(n=sample_size, random_state=1) # random state 1, so samples are reproducible


lus = samples["lu (mm)"].values
wus = samples["hu (mm)"].values
hus = samples["wu (mm)"].values

mortar_rests = samples["mortar rests (%)"].values
colour_rests = samples["colour rests (%)"].values

brick_original_dimensions = str(df_batch["brick_original_dimensions (mm)"].iloc[0]).strip().split(', ')

brick_length = float(brick_original_dimensions[0])
brick_width = float(brick_original_dimensions[1])
brick_height = float(brick_original_dimensions[2])

print(df_batch.head(5))
print(brick_length, brick_width, brick_height)
print("LUs:", lus)
print("WUs:", wus)
print("HUs:", hus)
print("Mortar Rests (%):", mortar_rests)
print("Colour Rests (%):", colour_rests)

5_k-3_10-1
utf encoder :
   Unnamed: 0  brick_id  delivery_nr  brick_format  \
0        6780      6781            5  Reichsformat   
1        6781      6782            5  Reichsformat   
2        6797      6798            5  Reichsformat   
3        6816      6817            5  Reichsformat   
4        6817      6818            5  Reichsformat   

  brick_original_dimensions (mm) brick_origin  brick_distance (km)  age (y)  \
0                   250, 120, 65      Kempten                  123      117   
1                   250, 120, 65      Kempten                  123      117   
2                   250, 120, 65      Kempten                  123      117   
3                   250, 120, 65      Kempten                  123      117   
4                   250, 120, 65      Kempten                  123      117   

  colour name  turn_nr  mortar rests (%)  colour rests (%)   R   G   B  \
0   red-brown        1              0.60              0.01  81  19  13   
1   red-brown        1     

In [1269]:
# "Testing" the samples 

def value_scaler(brightness_rank):
    # normaised in area [0, 1]
    normalized_value = (brightness_rank - 1) / (k - 1)
    # skaled to [0.1, 10.0]
    scaled_value = 0.1 + normalized_value * (10.0 - 0.1)
    return scaled_value

def test_statistics(values):
    mean_value = np.mean(values)
    std_value = np.std(values, ddof=1)  # Bessel's correct.
    CoV = (std_value / mean_value) # this is aequivalent to V in Merkblatt
    min_value = np.min(values)
    max_value = np.max(values)
    return mean_value, std_value, CoV, min_value, max_value

hue_values = []
pitch_values = []

mortar_rests_values = []
colour_rests_values = []

dry_weight_values = []
density_values = []
UPV_values = []
WAC_values = []
IRA_values = []
RPV_values = []

for idx, sample in samples.iterrows() :
    lu = sample["lu (mm)"]
    wu = sample["wu (mm)"]
    hu = sample["hu (mm)"]
    mortar_rests = sample["mortar rests (%)"]
    colour_rests = sample["colour rests (%)"]
    
    volume = round(lu * wu * hu, 1)
    print(int(volume))
    volume_perfect = brick_length * brick_width * brick_height
    volume_percent = round(volume / volume_perfect, 2)

    # HUE
    hue_indicator = value_scaler(brightness_rank)
    hue_deviation = random.uniform(-1.5, 3.0) # these parameter influence the choice of hue
    hue = round(min(max(hue_indicator + hue_deviation, 0.1), 10.0), 1)
    print(f"hue = {hue}")

    # Dry Weight, Density
    dry_weight = round(
    random.randint(
        int(2000 + ((hue - 0.1) / 9.9) * 4400 - 150), 
        int(2000 + ((hue - 0.1) / 9.9) * 4400 + 150)
    ) * float(volume_percent), 1)
    
    #dry_weight = round(random.randint((4300 - 150), (4300 + 150)) * float(volume_percent), 1)
    density = round(dry_weight / volume * 10**6, 1)
    
    pitch_deviation = random.uniform(-1.5, 1.5)
    pitch = round(min(max(hue + pitch_deviation, 0.1), 10.0), 1)
    #print(f"pitch = {pitch}")

    # UPV
    upv_predef = (round(0.7 + ((hue - 0.1) / 9.9) * 2.8, 1)) 
    if random.random() < 0.1 :
        variation = random.uniform(-1.0, 1.0)
    else : 
        variation = random.uniform(-0.3, 0.3)
    
    upv_predef += variation
    UPV = round(max(min(upv_predef, 3.6), 0.6), 1)

    IRA = round(
    random.uniform(
        max(0.6, 5.2 - ((hue - 0.1) / 9.9) * 4.2) - 0.3, 
        max(0.6, 5.2 - ((hue - 0.1) / 9.9) * 4.2) + 0.3
    ), 1)  # values as in table 1 of WAC_E-3-20.pdf

    WAC = round(
    random.uniform(
        max(1.0, 36.0 - ((hue - 0.1) / 9.9) * 35.0) - 5.0, 
        max(1.0, 36.0 - ((hue - 0.1) / 9.9) * 35.0) + 5.0
    ), 1)  # values as in table 1 of WAC_E-3-20.pdf
    
    if hue >= 8.0:
        RPV = random.uniform(0.400, 0.800)
    elif 5.0 <= hue < 8.0 :
        RPV = random.uniform(0.650, 0.900)
    elif 3.0 <= hue < 5.0 :
        RPV = random.uniform(0.760, 0.999)
    else:  # if hue is < 3.0
        RPV = random.uniform(0.900, 0.999)

    print(f"UPV = {UPV}")
    #print(f"IRA = {IRA}")
    #print(f"RPV = {RPV}")

    hue_values.append(hue)
    pitch_values.append(pitch)
    mortar_rests_values.append(mortar_rests)
    colour_rests_values.append(colour_rests)
    dry_weight_values.append(dry_weight)
    density_values.append(density)
    UPV_values.append(UPV)
    WAC_values.append(WAC)
    IRA_values.append(IRA)
    RPV_values.append(RPV)

hue_mean, hue_std, hue_CoV, hue_min, hue_max = test_statistics(hue_values)
pitch_mean, pitch_std, pitch_CoV, pitch_min, pitch_max = test_statistics(pitch_values)

mortar_rests_mean, mortar_rests_std, mortar_rests_CoV, mortar_rests_min, mortar_rests_max = test_statistics(mortar_rests_values)
colour_rests_mean, colour_rests_std, colour_rests_CoV, colour_rests_min, colour_rests_max = test_statistics(colour_rests_values)

dry_weight_mean, dry_weight_std, dry_weight_CoV, dry_weight_min, dry_weight_max = test_statistics(dry_weight_values)
density_mean, density_std, density_CoV, density_min, density_max = test_statistics(density_values)

UPV_mean, UPV_std, UPV_CoV, UPV_min, UPV_max = test_statistics(UPV_values)
WAC_mean, WAC_std, WAC_CoV, WAC_min, WAC_max = test_statistics(WAC_values)
IRA_mean, IRA_std, IRA_CoV, IRA_min, IRA_max = test_statistics(IRA_values)
RPV_mean, RPV_std, RPV_CoV, RPV_min, RPV_max = test_statistics(RPV_values)

print("")
print(f"result from the tests with n = {sample_size} : ")
print("")
print(f"hue, mean = {hue_mean:.2f}, std = {hue_std:.2f}, CoV = {hue_CoV:.2f}")
print(f"pitch, mean = {pitch_mean:.2f}, std = {pitch_std:.2f}, CoV = {pitch_CoV:.2f}")
print("")
print(f"mortar rests, mean = {mortar_rests_mean:.2f}, std = {mortar_rests_std:.2f}, CoV = {mortar_rests_CoV:.2f}")
print(f"colour rests, mean = {colour_rests_mean:.2f}, std = {colour_rests_std:.2f}, CoV = {colour_rests_CoV:.2f}")
print("")
print(f"Dry Weight, mean = {dry_weight_mean:.2f}, std = {dry_weight_std:.2f}, CoV = {dry_weight_CoV:.2f}")
print(f"Density, mean = {density_mean:.2f}, std = {density_std:.2f}, CoV = {density_CoV:.2f}")
print("")
print(f"UPV, mean = {UPV_mean:.2f}, std = {UPV_std:.2f}, CoV = {UPV_CoV:.2f}, min_value = {UPV_min}, max_value = {UPV_max}")
print(f"IRA, mean = {IRA_mean:.2f}, std = {IRA_std:.2f}, CoV = {IRA_CoV:.2f}, min_value = {IRA_min}, max_value = {IRA_max}")
print(f"RPV, mean = {RPV_mean:.3f}, std = {RPV_std:.3f}, CoV = {RPV_CoV:.3f}, min_value = {RPV_min: .3f}, max_value = {RPV_max : .3f}")


2135777
hue = 10.0
UPV = 3.2
2043550
hue = 10.0
UPV = 3.6
2017648
hue = 10.0
UPV = 3.4
2006495
hue = 9.5
UPV = 3.6
1921706
hue = 10.0
UPV = 3.2
1872069
hue = 10.0
UPV = 3.3

result from the tests with n = 6 : 

hue, mean = 9.92, std = 0.20, CoV = 0.02
pitch, mean = 9.57, std = 0.56, CoV = 0.06

mortar rests, mean = 0.19, std = 0.14, CoV = 0.78
colour rests, mean = 0.04, std = 0.05, CoV = 1.20

Dry Weight, mean = 6553.00, std = 369.91, CoV = 0.06
Density, mean = 3276.38, std = 64.90, CoV = 0.02

UPV, mean = 3.38, std = 0.18, CoV = 0.05, min_value = 3.2, max_value = 3.6
IRA, mean = 0.97, std = 0.20, CoV = 0.20, min_value = 0.8, max_value = 1.2
RPV, mean = 0.585, std = 0.068, CoV = 0.116, min_value =  0.464, max_value =  0.658


In [1270]:
"""
Linear interpolation to calculate the Compressive Strength from the UPV test results

two methods can be taken for this :
        - based on data by Räsänen, exponential function 
        - based on data by Dizhur, linear function (here used)
"""
# numbers taken from Dizhur et al. :
x1 = 0.7 # lowest UPV 
y1 = 10 # lowest MPa

x2 = 2.2 # highest UPV
y2 = 40 # highest MPa

def calc_m_c(x1, y1, x2, y2) :
    # y = m*x+c
    m = (y2 - y1) / (x2 - x1)  
    c = y1 - m * x1           
    return m, c

def upv_to_mpa_linear(upv, m, c):
    mpa = m * upv + c
    return max(mpa, 0)

# calcualte values for m and c
m, c = calc_m_c(x1, y1, x2, y2)

compressive_strength_values = []
for upv in UPV_values :
    compressive_strength = round(upv_to_mpa_linear(upv, m, c), 1)
    compressive_strength_values.append(compressive_strength)
    print(f"Test Result UPV : {upv} km/s, Compressive Strength : {upv_to_mpa_linear(upv, m, c):.3f} MPa")

#print(compressive_strength_values)

compressive_strength_mean, compressive_strength_std, compressive_strength_CoV, compressive_strength_min, compressive_strength_max = test_statistics(compressive_strength_values)

# Calculate Log Values for Compressive Strnength 

def log_statistics(values) :
    log_values = np.log(values)
    log_mean = np.mean(log_values)
    log_std = np.std(log_values, ddof=1)  # Bessel's correction ddof=1
    return log_mean, log_std

cs_log_mean, cs_log_std= log_statistics(compressive_strength_values)

print(compressive_strength_mean)
print(compressive_strength_CoV)
print(cs_log_mean, cs_log_std)
print("")
print(f"Compressive strength of {sample_size} test samples between {compressive_strength_min} N/mm2 and {compressive_strength_max} N/mm2 ")

Test Result UPV : 3.2 km/s, Compressive Strength : 60.000 MPa
Test Result UPV : 3.6 km/s, Compressive Strength : 68.000 MPa
Test Result UPV : 3.4 km/s, Compressive Strength : 64.000 MPa
Test Result UPV : 3.6 km/s, Compressive Strength : 68.000 MPa
Test Result UPV : 3.2 km/s, Compressive Strength : 60.000 MPa
Test Result UPV : 3.3 km/s, Compressive Strength : 62.000 MPa
63.666666666666664
0.057639199767635124
4.15228700053353 0.05732025986472031

Compressive strength of 6 test samples between 60.0 N/mm2 and 68.0 N/mm2 


In [1271]:
k5_n = 2.18  # Wert aus Tabelle 5 für n=6 und Vx unbekannt, WTA E-7-4

# 5 % Quantil
# fst5 = exp(my - k5_n * sy)
fst5 = np.exp(cs_log_mean - k5_n * cs_log_std)
print(round(fst5, 2))

fst = fst5 / 0.8
print(fst)

fst_final = min(fst, compressive_strength_mean)

print(f"Druckfestigkeit der Bestandsziegel von : fst = {fst_final : .2f}")

56.11
70.13853693909083
Druckfestigkeit der Bestandsziegel von : fst =  63.67


In [1272]:
# Druckfestigkeit des Mauerwerks mit historischem Kalkmörtel MGII / M 2.5 :
fm = 2.5

# Formel nach EC6 (3.6.1.2) fk = K * fb**a * fm**b, wobei fb = fst

if fst >= 45.0 :
    fb = 45.0
else :
    fb = fst
print(fb)

# K, a, b aus Tabelle NA.6 EN 1996-1-1_NA
K = 0.95
a = 0.585
b = 0.162

fk = K * fb**a * fm**b

print(f"Druckfestigkeit des Mauerwerks (mit historischem Kalkmörtel MG II), fk  = {fk : .2f}")

45.0
Druckfestigkeit des Mauerwerks (mit historischem Kalkmörtel MG II), fk  =  10.22


In [1273]:
# Add these values to the batch DataFrame
df_batch["test group ident"] = name
df_batch["hue (0.1 - 10.0)"] = round(hue_mean, 1)
df_batch["pitch (0.1 - 10.0)"] = round(pitch_mean, 1)

df_batch["samples mortar rests (%)"] = round(mortar_rests_mean, 2)
df_batch["samples colour rests (%)"] = round(colour_rests_mean, 2)

df_batch["volume (mm3)"] = int(volume)
df_batch["dry_weight (g)"] = round(dry_weight_mean, 1)
df_batch["density (kg/m3)"] = round(density_mean, 1)

df_batch["UPV (km/s)"] = round(UPV_mean, 1)
df_batch["fst (N/mm2)"] = round(fst_final, 2)
df_batch["fk (N/mm2)"] = round(fk, 2)

df_batch["WAC (%)"] = round(WAC_mean, 1)
df_batch["IRA (kg/(m2*min))"] = round(IRA_mean, 1)
df_batch["RPV (%)"] = round(RPV_mean, 3)

output_directory = "C:\\Users\\david\\OneDrive\\Documents\\01_Architektur\\02_MA Architektur\\TUM\\2024_Sommersemester\\01_THESIS\\02_MA_Thesis\\02_Listing\\01_Visual_Studio\\240816_Simulation\\02-3_Simulate_Tests\\01_Tested"
output_file = f"{date}_{name}_tested.csv"
#output_json = f"{date}_{name}_tested.json"

output_path = os.path.join(output_directory, output_file)
df_batch.to_csv(output_path, index=False)
#df_batch.to_json(output_json, orient='records', lines=True)

print(f"name of file : {output_file}")

name of file : 240826_5_k-3_10-1_tested.csv
